<a href="https://colab.research.google.com/github/kate-markina2709/Lab/blob/main/KA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import math
from random import randint


class El_Point:
    def __init__(self, val_x, val_y):
        self.val_x = val_x
        self.val_y = val_y


# проверка простоты числа
def Is_Prime(v_mod):
    if v_mod == 2 or v_mod == 3:
        return True
    if v_mod % 2 == 0 or v_mod < 2:
        return False
    for i in range(3, int(v_mod ** 0.5) + 1, 2):
        if v_mod % i == 0:
            return False
    return True


# обратное мультипликативное
def E_Algorithm(v1, v2):
    if v1 == 0:
        return v2, 0, 1
    else:
        g, y, x = E_Algorithm(v2 % v1, v1)
        return g, x - (v2 // v1) * y, y


def Inv_Mul(v_p, mod):
    g, x, y = E_Algorithm(v_p, mod)
    if g != 1:
        print('Обратного элемента не существует')
        return -1
    else:
        return x % mod


# задание основания
def Prime_mod():
    while True:
        p_mod = randint(500, 150000)
        if Is_Prime(p_mod):
            return p_mod


# проверка на удовлетворение условию 4a^3 + 27b^2
def AB_Value(mod):
    flag_c = False
    start_a = 0
    start_b = 0
    ab = []
    while flag_c is False:
        start_a = randint(1, mod)
        start_b = randint(1, mod)
        control = 4 * math.pow(start_a, 3) + 27 * math.pow(start_b, 2)
        if control % mod != 0:
            flag_c = True
    ab.append(start_a)
    ab.append(start_b)
    return ab


def Legendre_S(a, p):
    ls = pow(a, (p - 1)//2, p)
    if ls == p - 1:
        return -1
    return ls


def Sqrt_Mod(a, p):
    a %= p
    if a == 0:
        return [0]
    if p == 2:
        return [a]
    if Legendre_S(a, p) != 1:
        return []
    if p % 4 == 3:
        x = pow(a, (p + 1)//4, p)
        return [x, p-x]
    q, s = p - 1, 0
    while q % 2 == 0:
        s += 1
        q //= 2
    z = 1
    while Legendre_S(z, p) != -1:
        z += 1
    c = pow(z, q, p)
    x = pow(a, (q + 1)//2, p)
    t = pow(a, q, p)
    m = s
    while t != 1:
        i, e = 0, 2
        for i in range(1, m):
            if pow(t, e, p) == 1:
                break
            e *= 2
        b = pow(c, 1 << (m - i - 1), p)
        x = (x * b) % p
        t = (t * b * b) % p
        c = (b * b) % p
        m = i
    return [x, p-x]


# генерация точки P
def Point_Curve_P(p_a, p_b, mod):
    while True:
        v_x = randint(0, mod)
        tmp = (math.pow(v_x, 3) + p_a * v_x + p_b) % mod
        val_y = Sqrt_Mod(int(tmp), mod)
        if len(val_y) != 0 and val_y[0] == 0:
            return El_Point(v_x, 0)
        elif len(val_y) == 2:
            return El_Point(v_x, val_y[0])


# деление группы на 3 промежутка
def Div_Group(mod):
    flag_mas = []
    if mod % 3 == 0 or mod % 3 == 1:
        flag_mas.append(int(mod / 3))
        flag_mas.append(2 * int(mod / 3))
        flag_mas.append(mod)
    else:
        flag_mas.append(int(mod / 3))
        flag_mas.append(2 * int(mod / 3) + 1)
        flag_mas.append(mod)
    return flag_mas


# удвоение точки
def Double_of_Points(mod, p_p, val_a):
    y1 = p_p.val_y * 2
    if y1 == 0:
        point_3 = El_Point(0, math.inf)
        return point_3
    x1 = p_p.val_x
    vr = Inv_Mul(y1, mod)
    if vr == -1:
        return -1
    y1 = vr
    lam = (3 * math.pow(x1, 2) + val_a) * y1
    lam = lam % mod
    x3 = int(math.pow(lam, 2) - 2 * x1)
    x3 = x3 % mod
    y3 = int(lam * (x1 - x3) - p_p.val_y)
    y3 = y3 % mod
    point_3 = El_Point(x3, y3)
    return point_3


# сумма точек
def Sum_of_Points(mod, p_p, q_p, a):
    tmp_point = El_Point(0, 0)
    if p_p.val_x == q_p.val_x and p_p.val_y == q_p.val_y:
        tmp_point = Double_of_Points(mod, p_p, a)
        return tmp_point
    elif p_p.val_x == 0 and q_p.val_x == 0 and p_p.val_y == math.inf and q_p.val_y == math.inf:
        point_3 = El_Point(0, math.inf)
        return point_3
    elif p_p.val_x == 0 and p_p.val_y == math.inf:
        point_3 = El_Point(q_p.val_x, q_p.val_y)
        return point_3
    elif q_p.val_x == 0 and q_p.val_y == math.inf:
        point_3 = El_Point(p_p.val_x, p_p.val_y)
        return point_3
    elif p_p.val_y == -1 * q_p.val_y:
        point_3 = El_Point(0, math.inf)
        return point_3
    tmp_point.val_x = q_p.val_x - p_p.val_x
    if tmp_point.val_x == 0:
        point_3 = El_Point(0, math.inf)
        return point_3
    vr = tmp_point.val_x % mod
    tmp_point.val_x = Inv_Mul(vr, mod)
    if tmp_point.val_x == -1:
        return -1
    tmp_point.val_y = q_p.val_y - p_p.val_y
    lam = tmp_point.val_y * tmp_point.val_x
    lam = lam % mod
    x3 = int(math.pow(lam, 2) - p_p.val_x - q_p.val_x)
    x3 = x3 % mod
    y3 = int(lam * (p_p.val_x - x3) - p_p.val_y)
    y3 = y3 % mod
    point_3 = El_Point(x3, y3)
    return point_3


# степень
def Power_of_Point(mod, point, v_m, val_a):
    if v_m == 0:
        return El_Point(0, math.inf)
    elif v_m == 1:
        return point
    tmp_point = Double_of_Points(mod, point, val_a)
    if tmp_point == -1:
        return -1
    if v_m == 2:
        return tmp_point
    i = 2
    #print(v_m)
    while i < v_m:
        if point.val_x == tmp_point.val_x and point.val_y == tmp_point.val_y:
            tmp_point = Double_of_Points(mod, point, val_a)
            if tmp_point == -1:
                return -1
        else:
            tmp_point = Sum_of_Points(mod, tmp_point, point, val_a)
            if tmp_point == -1:
                return -1
        i = i + 1
    return tmp_point


# поиск порядка
def Num_Order(mod, point, val_a):
    max_r = 2 * math.sqrt(mod) + mod + 1
    rat = int(math.sqrt(max_r) + 1)
    list_p = []
    tmp_1 = point
    for i in range(0, rat):
        list_p.append([i + 1, tmp_1])
        tmp = Sum_of_Points(mod, point, tmp_1, val_a)
        if tmp == -1:
            return -1
        tmp_1 = tmp
    par_al = El_Point(list_p[-1][1].val_x, list_p[-1][1].val_y)
    y = -1 * par_al.val_y
    par_al.val_y = y % mod
    par_p = El_Point(0, math.inf)
    while True:
        for i in range(rat):
            for j in range(0, rat):
                if list_p[j][1].val_x == par_p.val_x and list_p[j][1].val_y == par_p.val_y:
                    return rat * i + list_p[j][0]
                if list_p[j][1].val_x == 0 and list_p[j][1].val_y == math.inf:
                    return list_p[j][0]
            par_p = Sum_of_Points(mod, par_p, par_al, val_a)
            if par_p == -1:
                return -1


# алгоритм поиска
def Alg_Work(mod_start, mod_order, p_a, iteration, p_p, q_p, flag_mas, control):
    a1 = -1
    b1 = -1
    it = 0
    i = 0
    if control == 0:
        start = int(input('Хотите ввести значения параметров для стартовой точки?\n 1. Да\n 2. Нет\n'))
        if start == 1:
            while a1 == -1 or a1 < 0 or a1 > mod_order:
                a1 = int(input('Введите значение параметра а: \n'))
                b1 = int(input('Введите значение параметра b: \n'))
        else:
            a1 = randint(1, mod_order - 1)
            b1 = randint(1, mod_order - 1)
    else:
        a1 = randint(1, mod_order - 1)
        b1 = randint(1, mod_order - 1)
    a2 = a1
    b2 = b1
    v1 = Power_of_Point(mod_start, p_p, a1, p_a)
    v2 = Power_of_Point(mod_start, q_p, b1, p_a)
    if v1 == -1 or v2 == -1:
        return -1
    x1 = Sum_of_Points(mod_start, v1, v2, p_a)
    if x1 == -1:
        return -1
    x2 = x1
    while True:
        it = it + 1
        if it > iteration and iteration != -1:
            return -2
        if x1.val_x >= 0 and x1.val_x < flag_mas[0]:
            v1 = x1
            x1 = Sum_of_Points(mod_start, q_p, v1, p_a)
            if x1 == -1:
                return -1
            v1 = a1
            a1 = v1 % mod_order
            v1 = b1
            b1 = (v1 + 1) % mod_order
        elif x1.val_x >= flag_mas[0] and x1.val_x < flag_mas[1]:
            v1 = x1
            x1 = Double_of_Points(mod_start, v1, p_a)
            if x1 == -1:
                return -1
            v1 = a1
            a1 = (2 * v1) % mod_order
            v1 = b1
            b1 = (2 * v1) % mod_order
        elif x1.val_x >= flag_mas[1] and x1.val_x < flag_mas[2]:
            v1 = x1
            x1 = Sum_of_Points(mod_start, p_p, v1, p_a)
            if x1 == -1:
                return -1
            v1 = a1
            a1 = (v1 + 1) % mod_order
            v1 = b1
            b1 = v1 % mod_order
        a = a2
        b = b2
        tmp = x2
        for k in range(0, 2):
            if tmp.val_x >= 0 and tmp.val_x < flag_mas[0]:
                v2 = tmp
                tmp = Sum_of_Points(mod_start, q_p, v2, p_a)
                if tmp == -1:
                    return -1
                v2 = b
                b = (v2 + 1) % mod_order
            elif tmp.val_x >= flag_mas[0] and tmp.val_x < flag_mas[1]:
                v2 = tmp
                tmp = Double_of_Points(mod_start, v2, p_a)
                if tmp == -1:
                    return -1
                v2 = a
                a = (2 * v2) % mod_order
                v2 = b
                b = (2 * v2) % mod_order
            elif tmp.val_x >= flag_mas[1] and tmp.val_x < flag_mas[2]:
                v2 = tmp
                tmp = Sum_of_Points(mod_start, p_p, v2, p_a)
                if tmp == -1:
                    return -1
                v2 = a
                a = (v2 + 1) % mod_order
        x2 = tmp
        a2 = a
        b2 = b
        i = i + 1
        if x1.val_x == x2.val_x and x1.val_y == x2.val_y:
            break
    if b1 != b2:
        v_a = a2 - a1
        v_b = b1 - b2
        a_res = v_a % mod_order
        b_res = v_b % mod_order
        d = math.gcd(b_res, mod_order)
        if d > 1:
            m_order = int(mod_order / d)
            b_end = int(b_res / d)
            a_end = int(a_res / d)
            b_end = Inv_Mul(b_end, m_order)
            if b_end == -1:
                return -1
            x = []
            v_1 = (a_end * b_end) % m_order
            for i in range(d):
                x.append(v_1 + i * m_order)
            for j in range(d):
                tmp_1 = Power_of_Point(mod_start, p_point, x[j], p_a)
                if tmp_1 == -1:
                    return -1
                if tmp_1.val_x == q_point.val_x and tmp_1.val_y == q_point.val_y:
                    return x[j]
        else:
            b_end = b_res
            b_res = Inv_Mul(b_end, mod_order)
            if b_res == -1:
                return -1
            res = (a_res * b_res) % mod_order
            return res
    else:
        return 0


# итоговый результат
def Find_Result(fl, mod_st, mod_or, a, it, p_p, q_p, v_check):
    res = 0
    while res == 0 or res == None:
        if v_check == 0:
            res = Alg_Work(mod_st, mod_or, a, it, p_p, q_p, fl, v_check)
            if res == -2:
                print('За указаное кол-во итераций результат не был найден\n')
                return 0
        else:
            res = Alg_Work(mod_st, mod_or, a, it, p_p, q_p, fl, v_check)
    return res


if __name__ == '__main__':
    print('Решение задачи ECDLP p0-методом Полларда')
    flag = int(input('Хотите протестировать программу?\n 1. Да \n 2. Нет\n'))
    test = 0
    check = 0
    file_result = open('test.txt', 'w')
    if flag == 1:
        test = int(input('Введите количество тестов?\n'))
        if test == 0 or test < 0:
            exit(0)
    while True:
        val_result = 0
        val_check = 0
        val_mod = Prime_mod()
        list_ab_e_c = AB_Value(val_mod)
        # стартовые параметры а и b
        a_ec = list_ab_e_c[0]
        b_ec = list_ab_e_c[1]
        p_point = El_Point(0, 0)
        val_order = 0
        while True:
            if val_order < 150:
                p_point = Point_Curve_P(a_ec, b_ec, val_mod)
                val_order = Num_Order(val_mod, p_point, a_ec)
            else:
                break
        q_point = Power_of_Point(val_mod, p_point, randint(15, val_order), a_ec)
        # определение окончания 3х интервалов
        end_ent = Div_Group(val_mod)
        if flag == 2:
            print(val_mod)
            print(list_ab_e_c[0], list_ab_e_c[1])
            print('Т-ка Р', p_point.val_x, p_point.val_y)
            print('Т-ка Q', q_point.val_x, q_point.val_y)
            # поиск решения задачи
            result = 0
            val = int(input('Хотите ввести предельное количество итераций?\n 1. Да\n 2. Нет\n'))
            if val == 2:
                result = Find_Result(end_ent, val_mod, val_order, a_ec, -1, p_point, q_point, 0)
            else:
                v = int(input('Введите количество итераций: '))
                while v < 0:
                    v = int(input('Введите количество итераций: '))
                result = Find_Result(end_ent, val_mod, val_order, a_ec, v, p_point, q_point, 0)
            print('Результат работы программы', result)
            check_val = Power_of_Point(val_mod, p_point, result, a_ec)
            if check_val.val_x == q_point.val_x and check_val.val_y == q_point.val_y:
                print('Oк')
            else:
                print('Ошибка')
            exit(0)
        else:
            result = Find_Result(end_ent, val_mod, val_order, a_ec, -1, p_point, q_point, 1)
            val_result = result
        # проверка
        check_val = Power_of_Point(val_mod, p_point, result, a_ec)
        if check_val.val_x == q_point.val_x and check_val.val_y == q_point.val_y:
            val_check = 0
        else:
            val_check = 1
        file_result.write('Значения параметров a, b: ' + str(a_ec) + ',' + str(b_ec))
        file_result.write('\n')
        file_result.write('Значение модуля: ' + str(val_mod))
        file_result.write('\n')
        file_result.write('Точка Р: ' + '[' + str(p_point.val_x) + ';' + str(p_point.val_y) + ']')
        file_result.write('\n')
        file_result.write('Точка Q: ' + '[' + str(q_point.val_x) + ';' + str(q_point.val_y) + ']')
        file_result.write('\n')
        file_result.write('Полученный результат: ' + str(val_result))
        file_result.write('\n\n')
        file_result.write('РЕЗУЛЬТАТ: ' + str(val_check))
        file_result.write('\n\n\n')
        print(check)
        check = check + 1
        if check == test:
            break
